# Milestone 3 - Time Series Forecasting

In [3]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML
import plotly.express as px
import plotly.tools as tls
import folium
import plotly.graph_objects as go
import ipywidgets as widgets
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import datetime
from dateutil.parser import parse as  date_parser
import numpy as np
import pandas as pd
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import threading
import time
import warnings
warnings.simplefilter('ignore')
%matplotlib inline

#### Importing daily timeseries Global dataset
  - Using pandas read_csv function to get dataset 

In [4]:
df = pd.read_csv (r'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

#### Droping Unnecessary column from dataset
  - Using df.drop to drop the columns

In [5]:
df=df.drop(columns=['Province/State','Lat','Long'])

#### Grouping the data by country
  - Using df.groupby to group the cells by country and perform sum

In [6]:
df=df.groupby(['Country/Region']).sum().copy()

In [7]:
countries=list(df.index.values)
countries.insert(0,'World')

### Prediction Function 
  - Country is provided as Argument
  - Train Test split is performed in the ratio 95:5 respectively
  ###### ARIMA
      - In arima function first the p,d,q are set from a range 0 to 8 
      - Best value of pdq is chosen by iterating and performing the model and comparing the results
      - The best value of pdq is supplied to the model
      - plot_predict() is used to forecast the value of confirmed cases in the coming month
      - Plot are shown using Plotly

In [8]:

def prediction_plots(country):

    def data(country):
        try:
            if(country=='World'):
                data=df.sum(axis = 0, skipna = True)
                data=pd.DataFrame(data)
                data.columns=['World']
            else:
                data=df.loc[country]
                data=pd.DataFrame(data)
        except:
            pass
        return data

    def mape(y1, y_pred): 
        y1, y_pred = np.array(y1), np.array(y_pred)
        return np.mean(np.abs((y1 - y_pred) / y1)) * 100

    def split(ts):
        #splitting 95%/5% because of little amount of data
        size = int(len(ts) * 0.95)
        train= ts[:size]
        test = ts[size:]
        return(train,test)


    #Arima modeling for ts
    def arima(ts,test):
        p=d=q=range(0,8)
        a=99999
        pdq=list(itertools.product(p,d,q))
        
        # PROGRESS BAR WIDGET
        
        progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, description='Loading: ')
        finished = False
        
        def work(progress):
            total = len(pdq)
            for i in range(total):
                if finished != True:
                    time.sleep(0.2)
                    progress.value = float(i+1)/len(pdq)
                else:
                    progress.value = len(pdq)
                    break

        thread = threading.Thread(target=work, args=(progress,))
        display(progress)
        # Start the progress bar thread
        thread.start()

        # Determining the best parameters
        for var in pdq:
            
            try:
                
                model = ARIMA(ts, order=var)
                result = model.fit()
                
                if (result.aic<=a) :
                    a=result.aic
                    param=var
                    
            except:
                continue
                
        model = ARIMA(ts, order=param)
        result = model.fit()
        finished = True

        # Forecast CURVE
        fig = result.plot_predict(start=int(len(ts) * 0.5), end=int(len(ts) * 1.2))
        fig.set_size_inches(12, 6)
        ax_list = fig.axes
        pred = result.forecast(steps=len(test))[0]    
#         for ax in ax_list:
#             ax.get_legend().remove()
#         plotly_fig= tls.mpl_to_plotly(fig)

#         legend = go.layout.Legend(
#             x=0.05,
#             y=0.95
#         )
#         plotly_fig.update_layout(title="Forecast: COVID 19 cases of " + country, xaxis_title='Date',
#         yaxis_title='No. of Confirmed Cases',
#         margin=dict(l=20, r=20, t=40, b=20),
#         paper_bgcolor="lightgrey",
#         width = 800, showlegend=True)
#         plotly_fig.show()
        
        # TEST VS PREDICTION Curve
        
        plotly_figs = go.Figure()
        plotly_figs.add_trace(go.Scatter(x=test.index.values, y=test[country],
                    mode='lines',
                    name='real values'))
        plotly_figs.add_trace(go.Scatter(x=test.index.values, y=pred,
                    mode='lines',
                    name='predictions'))
        
        
        plotly_figs.update_layout(title="Prediction : COVID 19 cases of " + country, xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800, showlegend=True
        )
        plotly_figs.show()

    train,test=split(data(country))

    arima(train,test)

    

#### Using Interactive Ipython widgets
  - Ipython widgets are used to make the response of arima function responsive according to user input.
  - Progress bar shows the progress of training process.

In [10]:
interact(prediction_plots, country=countries)
fig = go.FigureWidget( layout=go.Layout() )
ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Dropdown(description='country', options=('World', 'Afghanistan', 'Albania', 'Algeria', '…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …